In [1]:
from mxnet.gluon import nn
def mlpconv(channels,kernel_size,padding,strides=1,max_pooling=True):
    out=nn.Sequential() 
    with out.name_scope():
        out.add(
            nn.Conv2D(channels=channels,kernel_size=kernel_size,
                     strides=strides,padding=padding,activation='relu'),
            nn.Conv2D(channels=channels,kernel_size=1,
                     padding=0,strides=1,activation='relu'),
            nn.Conv2D(channels=channels,kernel_size=1,
                     padding=0,strides=1,activation='relu')
        )
        if max_pooling:
            out.add(nn.MaxPool2D(pool_size=3,strides=2))
    return out 

测试一下：

In [2]:

from mxnet import nd
blk=mlpconv(64,3,0)
blk.initialize()
x=nd.random.uniform(shape=(32,3,16,16))
y=blk(x) 
y.shape

(32, 64, 6, 6)

Nin 的卷积层的参数跟Alexnet类似，使用三组不同的设定 
- kernel: 11 * 11 ,channel;96
- kernel: 5 * 5 ,channels:256
- kernel:3 * 3 ,channels:384 
除了使用了1 * 1 卷积外，NiN 在最后不是使用全连接，而是使用通道数为输出类别个数的mlpconv,外接一个平均池化层来将每个通道里的数值平均层一个标量 。 


In [3]:
net=nn.Sequential()
with net.name_scope():
    net.add(
    mlpconv(96,11,0,strides=4),
    mlpconv(256,5,2),
    mlpconv(384,3,1),
        # 目标类为10类 
        nn.Dropout(.5),
        # 输出为batch_size * 10 * 5 * 5 ,通过AvgPool2D 转成 
        # batch_size * 10 * 1 * 1 
        mlpconv(10,3,1,max_pooling=False),
        nn.AvgPool2D(pool_size=5),
        # 转成 batch_size * 10 
        nn.Flatten()
    )

# 获取数据并训练
跟Alexnet类似，但使用了更大的学习率 



In [4]:
import sys 
sys.path.append('..') 
import utils 
from mxnet import gluon
from mxnet import init 
import time 
train_data,test_data=utils.load_data_fashion_mnist(batch_size=64,resize=224) 
ctx=utils.try_gpu() 
net.initialize(ctx=ctx,init=init.Xavier()) 
loss=gluon.loss.SoftmaxCrossEntropyLoss() 
trainer=gluon.Trainer(net.collect_params(),
                     'sgd',{'learning_rate':0.01})
start=time.time() 
utils.train(train_data,test_data,net,loss,
           trainer,ctx,num_epochs=1) 
end=time.time() 
print(" run time is : %f"%(end-start))

aaaaaaaa
Start training on  gpu(0)
Epoch 0. Loss: 2.290, Train acc 0.17, Test acc 0.18, Time 60.2 sec
 run time is : 60.242855
